In [4]:

import pandas as pd
    
import cx_Oracle

f=open("pwds.txt","r")
lines=f.readlines()
usr=lines[0]
pw=lines[1]
f.close()
path = 'C:\ORACLE\instantclient_19_11'

host = 'ALASTRIA'
port = 1524
service = 'CW1T2'
#cx_Oracle.init_oracle_client(lib_dir= path) # use this is service isn't initialised

cx_Oracle.clientversion()  
dsn_tns = cx_Oracle.makedsn(host, port, service_name=service)

conn = cx_Oracle.connect(user=usr, password=pw, dsn=dsn_tns)

c = conn.cursor()

query = """
select distinct "WBC_CASE_D"."CASE_NUM" "Case_Number", "WBC_OES_APPLICATION_D"."MOST_RECENT_WORK_EMPLOYER" "Employer_Name", "WBC_OES_APPLICATION_D"."MOST_RECENT_WORK_END_DATE" "End_Date", "WBC_OES_APPLICATION_D"."MOST_RECENT_WORK_INDUSTRY" "Industry", "WBC_OES_APPLICATION_D"."REASON_FOR_LEAVING_EMPLOYMENT" "Reason_for_Leaving", "WBC_OES_APPLICATION_D"."MOST_RECENT_WORK_START_DATE" "Start_Date", "WBC_OES_APPLICATION_D"."MOST_RECENT_WORK_JOB_TITLE" "Work_Title"
 from ((
select "WBC_OES_APPLICATION_F"."TAAAPA_WID" "TAAAPA_WID", "WBC_OES_APPLICATION_F"."CASE_WID" "CASE_WID"
 from "CDW"."WBC_OES_APPLICATION_F" "WBC_OES_APPLICATION_F"
) "WBC_OES_APPLICATION_F" INNER JOIN "CDW"."WBC_CASE_D" "WBC_CASE_D" on "WBC_OES_APPLICATION_F"."CASE_WID"="WBC_CASE_D"."CASE_WID") LEFT OUTER JOIN "CDW"."WBC_OES_APPLICATION_D" "WBC_OES_APPLICATION_D" on "WBC_OES_APPLICATION_F"."TAAAPA_WID"="WBC_OES_APPLICATION_D"."TAAAPA_WID"



"""

df = pd.read_sql(query, con=conn)


DatabaseError: ORA-12545: Connect failed because target host or object does not exist

In [5]:
from spellchecker import SpellChecker
import re
spell = SpellChecker()
text_file = open("Downloads/stopwords.txt", "r")
stopwords = text_file.read().split('\n')

custom_stop = ["agency","animal","autobody","automobile","beverage","bpo","business","call","camp","canadian","car","center","centre","commercial","community","control","course","culture","customer","development","employed","employment","equipment","fire","firm","foods","glass","group","home","house","housing","human","immigration","industry","information","interior","keeping","living","local","long","maintenance","natural","non","none","office","organization","pet","private","processing","production","protection","real","safety","sands","save","sector","self","senior","service","services","social","support","temp","term","tire","trade","trades","trading","various","video","windows","work","worker","works"]

stopwords.extend(custom_stop)
i_df = pd.read_csv(r'second_industry_matches_count_with_corrections.csv')
df = df[df['Industry'].notnull()]
for index, row in df.iterrows():

    x = re.findall(r"[\w']+", row['Industry'])
    x = [spell.correction(w) for w in x]
    df.at[index, 'Industry'] = ' '.join(x)

In [6]:
import warnings
warnings.filterwarnings('ignore')
df_out = df
df_out['Industry Code'] = "NA"
df_out['Industry Name'] = "NA"
df_out['combined'] = False
df_out = df_out[df_out['Industry'].notnull()]

n=0
for index, row in df_out.iterrows():
    n=n+1
    if n%1000 == 0:
        print(n)
        #break
    x = row['Industry'].split()
    x = [w for w in x if not w in stopwords]

    for xx in x:
        if row['combined'] == False:
            for tt in i_df['match word']:
                t = tt.split()
                t = [w for w in t if not w in stopwords]

                for z in t:
                    if xx.lower() == z.lower():
                        if row['combined'] == False:
                            row['combined'] = True
                            df_out.at[index, 'combined'] = True
                            df_out.at[index, 'Industry Code'] =  i_df.loc[i_df['match word'] == tt]["Industry Code"].item()
                            df_out.at[index, 'Industry Name'] =  i_df.loc[i_df['match word'] == tt]["Industry Name"].item()
                            
                            #df_out.loc[index, 'combined'] = True
                            #df_out.loc[index, 'Industry Code'] = i_df.loc[i_df['match word'] == tt]["Industry Code"]
                            #df_out.loc[index, 'Industry Name'] = i_df.loc[i_df['match word'] == tt]["Industry Name"]
                            #print(row)

df_out.head()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000


,Case_Number,Employer_Name,End_Date,Industry,Reason_for_Leaving,Start_Date,Work_Title,Industry Code,Industry Name,combined
1,0,Embers,None,Construction,None,2019-04-02 00:00:00,General Labourer,23,Construction,True
2,0,Akita Drilling LTD,2019-01-17 00:00:00,Oil field,"Very slow, Laid off, no work available, I am s...",1999-10-20 00:00:00,"Rig Manager, Safety Coach",21,"Mining, quarrying, and oil and gas extraction",True
3,0,Vince Hariott construction,None,construction site,I have done some trades work in construction o...,None,construction worker,23,Construction,True
4,0,super 8 motel,2018-08-15 00:00:00,hotel,Injury at work with surgery in December 2018 ...,2018-04-02 00:00:00,maintenance / security,72,Accommodation and food services,True
5,0,TELUS,2019-04-02 00:00:00,Telecommunications,None,2003-09-09 00:00:00,Network Technician,54,"Professional, scientific and technical services",True


In [7]:
df_out.to_excel('Downloads/new_industry_match_16_11_2021.xlsx')